Modified version of "k Means Clustering for Ch10 of Machine Learning in Action"
(original author: Peter Harrington)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def loadData(filename, sep, header): #data loader using pandas
    df = pd.read_table(filename,sep=sep,header=header)
    df = df.fillna(df.mean()) #set NaN values to mean
    return np.array(df) #convert to a numpy array (matrix)

In [ ]:
dataMat = loadData('testSet.txt',sep='\t',header=None) #delimiter is Tab, no header
print(dataMat.shape)

#### Note that we use row-wise representation today!

Exercise 2-1: implement standard k-means algorithm

In [ ]:
def randCent(dataSet, k):  #function to initialize seeds for each cluster
    n = np.shape(dataSet)[1]
    centroids = np.mat(np.zeros((k,n)))#create centroid mat
    for j in range(n):#create random cluster centers, within bounds of each dimension
        minJ = min(dataSet[:,j]) 
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = np.mat(minJ + rangeJ * np.random.rand(k,1))
    return centroids

In [ ]:
def kMeans(dataSet, k):
    m = np.shape(dataSet)[0] #number of data points
    clusterAssment = np.mat(np.zeros((m,2)))#create mat to assign data points 
                                      #0th column: index of the current cluster to which each data is assigned
                                      #1st column: distance to that cluster

    centroids = randCent(dataSet, k)  #initialize cluster centroids
    clusterChanged = True  #flag for termination (if false)
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = np.inf; minIndex = -1

            ###TODO: 
            # minIndex = ...    #index of the nearest centroid
            # minDist = ...  #Euclidean distance to the centroid
            
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        print(centroids)
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[np.nonzero(clusterAssment[:,0].A==cent)[0]]#get all the point in this cluster
            centroids[cent,:] = np.mean(ptsInClust, axis=0) #assign centroid to mean 
    return centroids, clusterAssment

In [ ]:
myCentroids, clustAssign = kMeans(dataMat,k=4)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(np.array(dataMat[:,0]),np.array(dataMat[:,1]),s=20,c='red',marker='s')
ax.scatter(np.array(myCentroids[:,0]),np.array(myCentroids[:,1]),s=20,c='green',marker='o')
plt.show()


Bonus: Bisecting k-means

In [ ]:
def biKmeans(dataSet, k):
    m = np.shape(dataSet)[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroid0 = np.mean(dataSet, axis=0).tolist()[0]
    centList =[centroid0] #create a list with one centroid
    for j in range(m):#calc initial Error
        clusterAssment[j,1] = np.linalg.norm(np.mat(centroid0)-dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = np.inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[np.nonzero(clusterAssment[:,0].A==i)[0],:]#get the data points currently in cluster i
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2) #call usual k-means internally
            sseSplit = sum(splitClustAss[:,1])#compare the SSE to the currrent minimum
            sseNotSplit = sum(clusterAssment[np.nonzero(clusterAssment[:,0].A!=i)[0],1])
            print("sseSplit, and notSplit: ",sseSplit,sseNotSplit)
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[np.nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList) #change 1 to 3,4, or whatever
        bestClustAss[np.nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        print('the bestCentToSplit is: ',bestCentToSplit)
        print('the len of bestClustAss is: ', len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]#replace a centroid with two best centroids 
        centList.append(bestNewCents[1,:].tolist()[0])
        clusterAssment[np.nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:]= bestClustAss#reassign new clusters, and SSE
    return np.mat(centList), clusterAssment

In [ ]:
myCentroids, clustAssign = biKmeans(dataMat,k=4)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(np.array(dataMat[:,0]),np.array(dataMat[:,1]),s=20,c='red',marker='s')
ax.scatter(np.array(myCentroids[:,0]),np.array(myCentroids[:,1]),s=20,c='green',marker='o')
plt.show()